In [105]:
import requests
from utils import load_config
import pandas as pd
from datetime import datetime
config = load_config()
HUBSPOT_API_KEY = config['hubspot']['api_key']
HUBSPOT_HEADERS = headers = {
    'Authorization': f'Bearer {HUBSPOT_API_KEY}',
    'Content-Type': 'application/json'
}

HUBSPOT_API_ROOT = 'https://api.hubapi.com'
HUBSPOT_API_CONTACTS = f'{HUBSPOT_API_ROOT}/crm/v3/objects/contacts'
HUBSPOT_URL_FEEDBACK_SURVEYS = f'{HUBSPOT_API_ROOT}/crm/v3/objects/feedback_submissions'


In [144]:
properties = ['hs_survey_name',
              'hs_survey_channel',
              'hs_sentiment',
              'hs_value',
              'hs_agent_name',
              'hs_content',
              'hs_source',
              'hs_survey_id', 
              'hs_survey_type',
              'hs_industry_standard_question_type',
              'hs_response',
              'hs_agent_email',
              'hs_submission_name',
              'hs_submission_url']
url_next = f"{HUBSPOT_URL_FEEDBACK_SURVEYS}?properties={','.join(properties)}"
print(url_next)
output = list()
while True:
    response = requests.get(url_next, headers=headers)
    res = response.json()
    output += res['results']
    if 'paging' not in res:
        break
    url_next = res['paging']['next']['link']

https://api.hubapi.com/crm/v3/objects/feedback_submissions?properties=hs_survey_name,hs_survey_channel,hs_sentiment,hs_value,hs_agent_name,hs_content,hs_source,hs_survey_id,hs_survey_type,hs_industry_standard_question_type,hs_response,hs_agent_email,hs_submission_name,hs_submission_url


In [145]:
nps_submissions = [o for o in output if o['properties']['hs_survey_type'] == 'NPS']
survey_names = pd.Series([nps['properties']['hs_survey_name'] for nps in nps_submissions]).value_counts()
survey_names

Enquête NPS Customer Success 2023    43
Enquête NPS Customer Success 2022    15
Enquête NPS janvier                   8
Enquête NPS février                   3
Enquête NPS mars                      3
Name: count, dtype: int64

In [146]:
nps_submissions

[{'id': '34483606768',
  'properties': {'hs_agent_email': None,
   'hs_agent_name': None,
   'hs_content': 'Très bon outil, très bonne pédagogie, facile à prendre en main et pertinent pour les apprenants ',
   'hs_createdate': '2021-02-02T17:32:35.272Z',
   'hs_industry_standard_question_type': None,
   'hs_lastmodifieddate': '2022-01-12T21:19:30.036Z',
   'hs_object_id': '34483606768',
   'hs_sentiment': 'Promoter',
   'hs_submission_name': 'Enquête NPS janvier - clemence.lafabriqueaneurones@gmail.com',
   'hs_submission_url': None,
   'hs_survey_channel': 'EMAIL',
   'hs_survey_id': '3',
   'hs_survey_name': 'Enquête NPS janvier',
   'hs_survey_type': 'NPS',
   'hs_value': '9'},
  'createdAt': '2021-02-02T17:32:35.272Z',
  'updatedAt': '2022-01-12T21:19:30.036Z',
  'archived': False},
 {'id': '34488518428',
  'properties': {'hs_agent_email': None,
   'hs_agent_name': None,
   'hs_content': None,
   'hs_createdate': '2021-02-02T19:32:54.252Z',
   'hs_industry_standard_question_type': 

In [147]:
tmp = [
        {
            'survey_name': nps['properties']['hs_survey_name'],
            'value': nps['properties']['hs_value'],
            'created_date': nps['properties']['hs_createdate'],
            'last_modified_date': nps['properties']['hs_lastmodifieddate'],
            'content': nps['properties']['hs_content'],
            'submission_name': nps['properties']['hs_submission_name'],
        } for nps in nps_submissions
    ]
df_nps = pd.DataFrame(tmp).sort_values('created_date', ascending=False)

In [150]:
import re

In [153]:
df_nps['submitter_email'] = [re.sub(nm_survey + ' -', '', nm_submission).strip() for (nm_survey, nm_submission) in zip(df_nps['survey_name'], df_nps['submission_name'])]

In [154]:
df_nps

,survey_name,value,created_date,last_modified_date,content,submission_name,submitter_email
71,Enquête NPS Customer Success 2023,9,2024-02-13T13:34:16.837Z,2024-02-13T13:38:47.717Z,Equipe très réactive pour faire progresser la ...,Enquête NPS Customer Success 2023 - isabelle.s...,isabelle.souvignet@monteiro-fr.com
70,Enquête NPS Customer Success 2023,10,2024-01-26T19:36:43.662Z,2024-01-26T19:39:03.635Z,Accompagnement \nLa facilité de conception \nL...,Enquête NPS Customer Success 2023 - catherine....,catherine.lenfant@bnpparibas.com
69,Enquête NPS Customer Success 2023,10,2024-01-22T05:18:51.427Z,2024-01-22T05:18:52.189Z,None,Enquête NPS Customer Success 2023 - ajohnson@m...,ajohnson@mcr-consultants.com
68,Enquête NPS Customer Success 2023,7,2024-01-19T16:48:10.285Z,2024-01-19T16:52:18.251Z,L'approche cognitive est très intéressante. Il...,Enquête NPS Customer Success 2023 - c-rondeau@...,c-rondeau@erg-sa.fr
67,Enquête NPS Customer Success 2023,5,2024-01-19T16:19:55.318Z,2024-01-19T16:19:59.865Z,None,Enquête NPS Customer Success 2023 - mverite@ed...,mverite@editions-foucher.fr
...,...,...,...,...,...,...,...
4,Enquête NPS janvier,10,2021-02-04T10:25:11.326Z,2023-09-19T09:10:59.709Z,None,Enquête NPS janvier - nplanty@lemon-adds.com,nplanty@lemon-adds.com
3,Enquête NPS janvier,10,2021-02-03T08:37:28.040Z,2022-01-11T16:25:30.435Z,Didask offre une solution digital a forte vale...,Enquête NPS janvier - joseph.krysmann@iecd.org,joseph.krysmann@iecd.org
2,Enquête NPS janvier,8,2021-02-03T06:05:30.042Z,2022-01-11T14:30:23.205Z,Entrer dans le déroulé de Didask n’est pas ais...,Enquête NPS janvier - cbmaud@hotmail.com,cbmaud@hotmail.com
1,Enquête NPS janvier,10,2021-02-02T19:32:54.252Z,2023-10-26T10:09:07.413Z,None,Enquête NPS janvier - juliettevincent@mjcoachi...,juliettevincent@mjcoaching.fr


In [114]:
rr

[{'survey_name': 'Enquête NPS janvier',
  'value': '9',
  'created_date': '2021-02-02T17:32:35.272Z',
  'last_modified_date': '2022-01-12T21:19:30.036Z'},
 {'survey_name': 'Enquête NPS janvier',
  'value': '10',
  'created_date': '2021-02-02T19:32:54.252Z',
  'last_modified_date': '2023-10-26T10:09:07.413Z'},
 {'survey_name': 'Enquête NPS janvier',
  'value': '8',
  'created_date': '2021-02-03T06:05:30.042Z',
  'last_modified_date': '2022-01-11T14:30:23.205Z'},
 {'survey_name': 'Enquête NPS janvier',
  'value': '10',
  'created_date': '2021-02-03T08:37:28.040Z',
  'last_modified_date': '2022-01-11T16:25:30.435Z'},
 {'survey_name': 'Enquête NPS janvier',
  'value': '10',
  'created_date': '2021-02-04T10:25:11.326Z',
  'last_modified_date': '2023-09-19T09:10:59.709Z'},
 {'survey_name': 'Enquête NPS janvier',
  'value': '10',
  'created_date': '2021-03-01T13:20:15.205Z',
  'last_modified_date': '2022-01-11T13:03:05.639Z'},
 {'survey_name': 'Enquête NPS février',
  'value': '10',
  'create

In [98]:
nps_scores = pd.Series([nps['properties']['hs_value'] for nps in nps_submissions])

In [103]:
e = [nps['properties']['hs_createdate'] for nps in nps_submissions][0]

In [108]:
datetime.strptime(e, '%Y-%m-%dT%H:%M:%S.%fZ')

2021

In [100]:
pd.DataFrame([(nps['properties']['hs_value'],) for nps in nps_submissions])

10    38
9     18
8      8
7      4
5      2
6      2
Name: count, dtype: int64

## Hubspot python SDK

In [2]:
from hubspot import HubSpot
#from hubspot.crm.extensions.feedback_submissions import ApiException

client = HubSpot(api_key=HUBSPOT_API_KEY)

In [11]:
survey_id = '342049645'

# Fetch feedback submissions for the given survey ID
# Adjust the limit as per your requirement, here it's set to 10 for demonstration
feedback_submissions = client.crm.objects.feedback_submissions.basic_api.get_page(
    limit=10, 
    feedback_survey_id=survey_id
)

# Assuming each submission contains an NPS score, let's print them
for submission in feedback_submissions.results:
    # The structure of your submission might vary, adjust the access accordingly
    # This is a generic placeholder to access the NPS score; the exact path might differ
    nps_score = submission.properties.get('nps_score')
    print(f"NPS Score: {nps_score}")


ApiTypeError: Got an unexpected keyword argument 'feedback_survey_id' to method get_page

In [12]:
help(client.crm.objects.feedback_submissions.basic_api.get_page)

Help on method get_page in module hubspot.crm.objects.feedback_submissions.api.basic_api:

get_page(**kwargs) method of hubspot.crm.objects.feedback_submissions.api.basic_api.BasicApi instance
    List  # noqa: E501

    Read a page of feedback submissions. Control what is returned via the `properties` query param.  # noqa: E501
    This method makes a synchronous HTTP request by default. To make an
    asynchronous HTTP request, please pass async_req=True

    >>> thread = api.get_page(async_req=True)
    >>> result = thread.get()

    :param limit: The maximum number of results to display per page.
    :type limit: int
    :param after: The paging cursor token of the last successfully read resource will be returned as the `paging.next.after` JSON property of a paged response containing more results.
    :type after: str
    :param properties: A comma separated list of the properties to be returned in the response. If any of the specified properties are not present on the requested ob

In [3]:
feedback_surveys_client = client.crm.objects.feedback_surveys

# Get the list of feedback surveys
# Note: As of my last update, direct support for feedback surveys might not be fully integrated into the Python client.
# This step may require adjusting based on the actual methods available in the latest version of the hubspot-api-client library.
surveys = feedback_surveys_client.get_all()

# Iterate through the surveys and print their details
for survey in surveys:
    print(f"Survey ID: {survey.id}, Name: {survey.name}")

AttributeError: 'Discovery' object has no attribute 'feedback_surveys'